In [22]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm_helper import node2pt, way2line, rltn2cltn, osm_container
import geopandas as gp
from osmread import Node, Way, Relation
import datetime

In [27]:
OSM_DC_BBOX_PATH = '../data/dc_bbox.osm'
OSM_DC_IDS_PATH = '../data/dc_osm_ids_within_polygon.txt'

In [3]:
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()

begin reading osm 2016-11-09 22:27:42.494000
finish reading osm 2016-11-09 22:28:54.894000
["len of <class 'osmread.elements.Way'> = 306332", "len of <class 'osmread.elements.Node'> = 2546260", "len of <class 'osmread.elements.Relation'> = 3427"]


In [5]:
DC_POLY_gpdf = gp.read_file('../data/dc_polygon.geojson')
DC_POLY = DC_POLY_gpdf.geometry.values[0]

In [14]:
def get_node_ids_given_polygon(nodes, polygon):
    print datetime.datetime.now()
    node_ids = set()
    for node in nodes:
        pt = node2pt(node)
        if pt.intersects(polygon):
            node_ids.add(node.id)
    print datetime.datetime.now()
    return node_ids

def get_way_ids_given_polygon(ways, polygon,osm_data):
    print datetime.datetime.now()
    way_ids = set()
    for way in ways:
        ln = way2line(osm_data, way)
        if ln.intersects(polygon):
            way_ids.add(way.id)
    print datetime.datetime.now()
    return way_ids

In [11]:
DC_NODE_IDS = get_node_ids_given_polygon(OSM_DC_BBOX.osm_objs[Node], DC_POLY)
DC_WAYS_IDS = get_way_ids_given_polygon(OSM_DC_BBOX.osm_objs[Way], DC_POLY, OSM_DC_BBOX)

2016-11-09 22:29:45.881000
2016-11-09 22:34:23.454000
2016-11-09 22:34:23.455000
2016-11-09 22:35:23.948000


In [25]:
def get_rltn_ids_given_polygon(rltns, polygon, osm_data):
    print datetime.datetime.now()
    rltn_ids = set()
    for rltn in rltns:
        try:
            shpcltn = rltn2cltn(osm_data,rltn)
        except Exception as e:
            continue
            
        for shptype, objs in shpcltn.items():
            for obj in objs:
                if obj.intersects(polygon):
                    rltn_ids.add(rltn.id)
    print datetime.datetime.now()
    return rltn_ids
    
DC_RLTN_IDS = get_rltn_ids_given_polygon(OSM_DC_BBOX.osm_objs[Relation], DC_POLY, OSM_DC_BBOX)

2016-11-09 22:41:17.957000
2016-11-09 22:41:24.855000


In [24]:
len(DC_WAYS_IDS), len(DC_NODE_IDS), len(DC_RLTN_IDS)

(194767, 1652906, 2556)

In [26]:
DC_OSM_IDS = {'Node':DC_NODE_IDS, 'Way':DC_WAYS_IDS, 'Relation': DC_RLTN_IDS}

In [29]:
with open(OSM_DC_IDS_PATH, 'w') as f:
    f.write(str(DC_OSM_IDS))